In [0]:
#import keras
# from keras.models import Sequential, Model, load_model
# from keras.layers import Dense, Conv2D, Input, Lambda, Flatten, Reshape, MaxPooling2D, Multiply, Activation, Add, Concatenate, BatchNormalization
# from keras.optimizers import Adam, SGD
# from keras.callbacks import EarlyStopping
# import keras.backend as K
# import tensorflow as tf
# from keras.engine.topology import Layer
# from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Conv2D, Input, Lambda, Flatten, Reshape, MaxPool2D, Multiply, Activation, Add, Concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
import h5py
import matplotlib.pyplot as plt
from google.colab import drive


In [2]:
from numpy.random import seed
seed(1)
print(tf.__version__)
tf.random.set_seed(0)

2.2.0-rc2


In [0]:
# from tensorflow.python.client import device_lib
#assert len(K.tensorflow_backend._get_available_gpus()) > 0
# assert 'GPU' in str(device_lib.list_local_devices())
physical_devices = tf.config.list_physical_devices('GPU') 
assert len(physical_devices) > 0
#assert len(K.tensorflow_backend._get_available_gpus()) > 0

In [4]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
%cd "drive/My Drive/BTP/code"
%ls

/content/drive/My Drive/BTP/code
Attention_train_v1.ipynb       extra/
Attention_train_v1-LTPA.ipynb  __init__.py
Attention_train_v2.ipynb       multi-head_attention_mnist_v1.ipynb
Attention_train_v2-LTPA.ipynb  multi-head_attention_mnist_v2.ipynb
Bateaux_code/                  multi-head_attention_mnist_v3.ipynb
bateaux_metrics_v1.py          playground.ipynb
bateaux_metrics_v2.py          __pycache__/
dataset_creation/              siamese.ipynb
evaluate_models.ipynb          Untitled0.ipynb


In [6]:
from bateaux_metrics_v2 import *

Using TensorFlow backend.


In [0]:
parent_dir = os.path.dirname(os.getcwd())
train_set_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'train_set.hdf5')
train_GT_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'train_GT.csv')
test_set_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'test_set.hdf5')
test_GT_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'test_GT.csv')

weight_file = os.path.join(parent_dir, 'models', 'Attention', '1-scene', 'att_model-LTPA_v3.hdf5')
info_file = os.path.join(parent_dir, 'models', 'Attention', '1-scene', 'att_model-LTPA_v3.txt')

In [0]:
img_width = 128
img_height = 128
channels = 3

In [0]:
#K.clear_session()
tf.keras.backend.clear_session()

In [0]:
 def VGGBlock(x, regularizer = None, batchnorm = False):
   if batchnorm:
      x = Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv1')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv2')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)

      x = Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv3')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv4')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)

      x = Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv5')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv6')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv7')(x)
      x = BatchNormalization()(x)
      local1 = Activation('relu')(x)  # batch*x*y*channel
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool1')(x)

      x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv8')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv9')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv10')(x)
      x = BatchNormalization()(x)
      local2 = Activation('relu')(x)
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool2')(local2)

      x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv11')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv12')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv13')(x)
      x = BatchNormalization()(x)
      local3 = Activation('relu')(x)
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool3')(local3)

      x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv14')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool4')(x)
      x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizer, name='conv15')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool5')(x)
      x = Flatten(name='pregflatten')(x)
      g = Dense(512, activation='relu', kernel_regularizer=regularizer, name='globalg')(x)  # batch*512
      return (g, local1, local2, local3)
   else:            
      x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv1')(x)
      x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv2')(x)

      x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv3')(x)
      x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv4')(x)

      x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv5')(x)
      x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv6')(x)
      local1 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv7')(x)  # batch*x*y*channel
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool1')(local1)

      x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv8')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv9')(x)
      local2 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv10')(x)
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool2')(local2)

      x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv11')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv12')(x)
      local3 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv13')(x)
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool3')(local3)

      x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv14')(x)
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool4')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer, name='conv15')(x)
      x = MaxPool2D((2, 2), strides=(2, 2), name='pool5')(x)
      x = Flatten(name='pregflatten')(x)
      g = Dense(512, activation='relu', kernel_regularizer=regularizer, name='globalg')(x)  # batch*512
      return (g, local1, local2, local3)


In [0]:
class ParametrisedCompatibility(Layer):
    def __init__(self, kernel_regularizer=None, **kwargs):
        super(ParametrisedCompatibility, self).__init__(**kwargs)
        self.regularizer = kernel_regularizer

    def build(self, input_shape):
        self.u = self.add_weight(name='u', shape=(input_shape[0][3], 1), initializer='uniform', regularizer=self.regularizer, trainable=True)
        super(ParametrisedCompatibility, self).build(input_shape)

    def call(self, x):  # add l and g. Dot the sum with u.
        return K.dot(K.map_fn(lambda lam: (lam[0]+lam[1]),elems=(x),dtype='float32'), self.u)

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], input_shape[0][2])

In [0]:
weight_decay=0.0005
regularizer = tf.keras.regularizers.l2(weight_decay)

In [0]:
inp = Input((img_width, img_height, channels), name='input_layer')

(g, local1, local2, local3) = VGGBlock(inp, regularizer=regularizer, batchnorm=True)

In [14]:
print(local1.get_shape)
print(local2.get_shape)
print(local3.get_shape)
print(g.get_shape)

<bound method Tensor.get_shape of <tf.Tensor 'activation_6/Identity:0' shape=(None, 128, 128, 256) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'activation_9/Identity:0' shape=(None, 64, 64, 512) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'activation_12/Identity:0' shape=(None, 32, 32, 512) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'globalg/Identity:0' shape=(None, 512) dtype=float32>>


In [0]:
# g = Flatten()(g)
# g = Dense(512, activation='relu')(g)
# print(g.get_shape)

In [16]:
l1 = Dense(512, kernel_regularizer=regularizer, name='l1connectordense')(local1)  # batch*x*y*512
c1 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc1')([l1, g])  # batch*x*y
flatc1 = Flatten(name='flatc1')(c1)  # batch*xy
a1 = Activation('softmax', name='softmax1')(flatc1)  # batch*xy

reshaped1 = Reshape((-1,512), name='reshape1')(l1)  # batch*xy*512.
g1 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g1')([a1, reshaped1])  # batch*512.

l2 = local2
c2 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc2')([l2, g])
flatc2 = Flatten(name='flatc2')(c2)
a2 = Activation('softmax', name='softmax2')(flatc2)

reshaped2 =  Reshape((-1,512), name='reshape2')(l2)
g2 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g2')([a2, reshaped2])

l3 = local3
c3 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc3')([l3, g])
flatc3 = Flatten(name='flatc3')(c3)
a3 = Activation('softmax', name='softmax3')(flatc3)

reshaped3 = Reshape((-1,512), name='reshape3')(l3)
g3 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g3')([a3, reshaped3])
glist = [g3, g2, g1]
predictedG = g3
predictedG = Concatenate(axis=1, name='ConcatG')(glist)
out = Dense(6, kernel_regularizer=regularizer, name='output')(predictedG)
model = Model(inputs=inp, outputs=out)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 1792        input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
epochs = 100
batch_size = 16
epochs_to_wait_for_improve = 3

In [0]:
lr = 0.01
momentum = 0
nesterov = False
#optimizer = SGD(lr=lr, momentum=momentum, nesterov=nesterov, decay=0.01)
optimizer = Adam(lr=0.01)
model.compile(optimizer=optimizer, loss=euclidean_loss(beta=beta), metrics=[euclidean_dist, x_error, y_error, z_error])
early_stopping_callback = EarlyStopping(monitor='val_euclidean_dist', patience=epochs_to_wait_for_improve, restore_best_weights=True)

In [19]:
h5_reader = h5py.File(train_set_file, 'r')
print(list(h5_reader.keys()))
y = np.loadtxt(train_GT_file, dtype=np.float32)
split_idx = int(y.shape[0] * 0.9)

x_train = np.array(h5_reader['train_img'][:split_idx], dtype=np.float32, copy=False) / 255
x_val = np.array(h5_reader['train_img'][split_idx:], dtype=np.float32, copy=False) / 255
y_train = y[:split_idx, :]
y_val = y[split_idx:, :]

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

h5_reader.close()

['train_img']
(23759, 128, 128, 3)
(23759, 6)
(2640, 128, 128, 3)
(2640, 6)


In [20]:
steps_per_epoch = x_train.shape[0] // batch_size
print(steps_per_epoch)

1484


In [0]:
img_gen = ImageDataGenerator()

In [0]:
model.fit_generator(img_gen.flow(x_train, y_train, batch_size=batch_size),
            steps_per_epoch=steps_per_epoch, validation_data=(x_val, y_val),
            epochs=epochs, callbacks=[early_stopping_callback])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
1484/1484 [==============================] - 2448s 2s/step - loss: 1.2665 - euclidean_dist: 0.0489 - x_error: 0.0358 - y_error: 0.0455 - z_error: 0.0444 - val_loss: 1.7042 - val_euclidean_dist: 0.3886 - val_x_error: 0.5010 - val_y_error: 0.3416 - val_z_error: 0.2891
Epoch 2/100
1484/1484 [==============================] - 2424s 2s/step - loss: 0.5552 - euclidean_dist: 0.0309 - x_error: 0.0271 - y_error: 0.0271 - z_error: 0.0269 - val_loss: 0.4784 - val_euclidean_dist: 0.0397 - val_x_error: 0.0537 - val_y_error: 0.0253 - val_z_error: 0.0215
Epoch 3/100
1372/1484 [==========================>...] - ETA: 2:55 - loss: 0.4547 - euclidean_dist: 0.0287 - x_error: 0.0248 - y_error: 0.0265 - z_error: 0.0239

In [0]:
model.save_weights(weight_file)
print('Saved trained model at %s ' % weight_file)

In [0]:
h5_reader = h5py.File(test_set_file, 'r')
#print(h5_reader.keys())
x_test = h5_reader['test_img']
x_test = np.array(x_test)
h5_reader.close()
x_test = x_test.astype('float32')
x_test /= 255

In [0]:
y_test = np.loadtxt(test_GT_file, dtype=np.float32)

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
metric_names = model.metrics_names

In [0]:
for metric, val in zip(metric_names, scores):
  print("{} : {}".format(metric, val))

In [0]:
file_obj = open(info_file, 'w')
file_obj.write("MODEL SUMMARY\n")
#summary = model.summary()
model.summary(print_fn=lambda x: file_obj.write(x + '\n'))
file_obj.write("\n")
file_obj.write("Optimizer : Adam - batch_size = {}\n\n".format(batch_size))
#file_obj.write("Optimizer : SGD - lr = {}, momentum = {}, batch_size = {}\n\n".format(lr, momentum, batch_size))

file_obj.write("\n\nPerformance")
for metric, val in zip(metric_names, scores):
  file_obj.write("{} : {}\n".format(metric, val))

file_obj.write("\n\nFILE PATHS\n")
file_obj.write("Training set : {}\n".format(train_set_file))
file_obj.write("Testing set : {}\n".format(test_set_file))
file_obj.write("Weight file : {}\n".format(weight_file))
file_obj.close()